## NYC Flood Vulnerability Index data

use NYC Flood Vulnerability Index data to get the flood risk score

In [1]:
# import packages
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

import geopandas as gpd

from shapely.geometry import Point

from haversine import haversine, Unit

from sklearn.preprocessing import MinMaxScaler

In [ ]:
flood_data = pd.read_csv('New_York_City_s_Flood_Vulnerability_Index_20250529.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [3]:
flood_data.head()

,the_geom,GEOID,FSHRI,FVI_storm_surge_present,FVI_storm_surge_2050s,FVI_storm_surge_2080s,FVI_tidal_2020s,FVI_tidal_2050s,FVI_tidal_2080s
0,"MULTIPOLYGON (((-73.860344 40.83329400000001, ...",36005009200,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,"MULTIPOLYGON (((-73.873954 40.83423700000001, ...",36005006400,5.0,NaN,NaN,5.0,NaN,NaN,NaN
2,"MULTIPOLYGON (((-73.888319 40.84517900000001, ...",36005036501,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,"MULTIPOLYGON (((-73.908225 40.816203, -73.9075...",36005007900,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,"MULTIPOLYGON (((-73.920958 40.855112, -73.9200...",36005024700,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
flood_data.dtypes

the_geom                    object
GEOID                        int64
FSHRI                      float64
FVI_storm_surge_present    float64
FVI_storm_surge_2050s      float64
FVI_storm_surge_2080s      float64
FVI_tidal_2020s            float64
FVI_tidal_2050s            float64
FVI_tidal_2080s            float64
dtype: object

In [ ]:
#filter GEOID for Manhatten (36061)
flood_manhattan = flood_data[flood_data["GEOID"].astype(str).str.startswith("36061")]


In [6]:
flood_manhattan.head()

,the_geom,GEOID,FSHRI,FVI_storm_surge_present,FVI_storm_surge_2050s,FVI_storm_surge_2080s,FVI_tidal_2020s,FVI_tidal_2050s,FVI_tidal_2080s
116,MULTIPOLYGON (((-73.98701500000001 40.76103799...,36061012500,1.0,NaN,NaN,NaN,NaN,NaN,NaN
117,"MULTIPOLYGON (((-73.969337 40.763546999999996,...",36061011402,1.0,NaN,NaN,NaN,NaN,NaN,NaN
118,"MULTIPOLYGON (((-73.989297 40.75790699999999, ...",36061011900,2.0,NaN,NaN,NaN,NaN,NaN,NaN
119,"MULTIPOLYGON (((-73.954482 40.77813900000001, ...",36061014601,1.0,NaN,NaN,NaN,NaN,NaN,NaN
120,"MULTIPOLYGON (((-73.940861 40.808334, -73.9404...",36061020600,2.0,NaN,1.0,2.0,NaN,NaN,NaN


In [7]:
flood_manhattan.shape

(300, 9)

In [8]:
missing_values = flood_manhattan.isna().sum()
print(missing_values)

the_geom                     0
GEOID                        0
FSHRI                        0
FVI_storm_surge_present    224
FVI_storm_surge_2050s      197
FVI_storm_surge_2080s      185
FVI_tidal_2020s            292
FVI_tidal_2050s            289
FVI_tidal_2080s            271
dtype: int64


In [9]:
flood_manhattan.dtypes

the_geom                    object
GEOID                        int64
FSHRI                      float64
FVI_storm_surge_present    float64
FVI_storm_surge_2050s      float64
FVI_storm_surge_2080s      float64
FVI_tidal_2020s            float64
FVI_tidal_2050s            float64
FVI_tidal_2080s            float64
dtype: object

In [10]:
#Missing values will be imputed with 0
flood_manhattan=flood_manhattan.fillna(0)

In [12]:
# Normalise present storm surge
scaler_ss = MinMaxScaler(feature_range=(1, 10))
ss_scaled = scaler_ss.fit_transform(flood_manhattan[["FVI_storm_surge_present"]])

#Normalize tidal flooding event for 50s
scaler_tid = MinMaxScaler(feature_range=(1, 10))
tid_scaled = scaler_tid.fit_transform(flood_manhattan[["FVI_tidal_2050s"]])

In [13]:
#creating flood risk score with weighted sum of normalized values
flood_manhattan["flood_risk_score"] = (
    0.6 * ss_scaled.flatten() +
    0.4 * tid_scaled.flatten()
)

In [16]:
flood_manhattan.head()

,the_geom,GEOID,FSHRI,FVI_storm_surge_present,FVI_storm_surge_2050s,FVI_storm_surge_2080s,FVI_tidal_2020s,FVI_tidal_2050s,FVI_tidal_2080s,flood_risk_score
116,MULTIPOLYGON (((-73.98701500000001 40.76103799...,36061012500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
117,"MULTIPOLYGON (((-73.969337 40.763546999999996,...",36061011402,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
118,"MULTIPOLYGON (((-73.989297 40.75790699999999, ...",36061011900,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
119,"MULTIPOLYGON (((-73.954482 40.77813900000001, ...",36061014601,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
120,"MULTIPOLYGON (((-73.940861 40.808334, -73.9404...",36061020600,2.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0


In [ ]:
#Export Results into csv
flood_manhattan.to_csv('manhatten_flood_risk_scores.csv', index=False)